<a href="https://colab.research.google.com/github/spaceo521/Data-Science-Projects/blob/main/Week8_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pulp
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

## First T20 match Ind vs SL

In [ ]:
df = pd.read_csv('ind_sl_t20i_1st_2022_02_24.csv')

In [ ]:
df = df.join(pd.get_dummies(df[['Role', 'Team']]))

In [ ]:
df.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1,0,0,0,1,1,0
1,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1,0,1,0,0,1,0
2,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1,0,0,1,0,1,0
3,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1,0,1,0,0,0,1
4,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1,0,1,0,0,1,0


In [ ]:
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

mo = pulp.LpProblem('Points maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Points'] for obj in df.index])

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) <= 7

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) <= 4

In [ ]:
mo.solve()

1

In [ ]:
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

In [ ]:
fdf = df[df['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

In [ ]:
fdf

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1,0,1,0,0,1,0,1
1,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1,0,1,0,0,0,1,1
2,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1,0,1,0,0,1,0,1
3,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1,0,0,0,1,1,0,1
4,3692,38632,V Iyer,10917,IND,2,AR,53.22,8.5,0,1,50,1,1,0,0,0,1,0,1
5,3692,38632,R Jadeja,587,IND,2,AR,74.73,9.0,0,1,34,1,1,0,0,0,1,0,1
6,3692,38632,D Shanaka,8422,SL,5,AR,50.75,9.0,0,1,32,1,1,0,0,0,0,1,1
7,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1,0,0,1,0,1,0,1
8,3692,38632,Y Chahal,7910,IND,2,BOWL,40.62,8.5,0,1,35,1,0,0,1,0,1,0,1
9,3692,38632,L Kumara,10928,SL,5,BOWL,34.56,8.5,0,1,35,1,0,0,1,0,0,1,1


In [ ]:
match1=sum(fdf.Points)
match1

653

points for 1st Match is 653

## Second T20 match Ind vs SL

In [ ]:
df = pd.read_csv('ind_sl_t20i_2nd_2022_02_26.csv')

In [ ]:
df = df.join(pd.get_dummies(df[['Role', 'Team']]))

In [ ]:
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

mo = pulp.LpProblem('Points maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Points'] for obj in df.index])

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) <= 7

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) <= 4

In [ ]:
mo.solve()

1

In [ ]:
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

In [ ]:
fdf = df[df['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

In [ ]:
fdf

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1,0,1,0,0,1,0,1
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1,0,1,0,0,0,1,1
2,3692,38637,D Gunathilaka,8387,SL,5,BAT,4.45,8.5,0,0,1,56,1,0,1,0,0,0,1,1
3,3692,38637,S Samson,8271,IND,2,WK,25.94,8.0,0,0,1,59,1,0,0,0,1,1,0,1
4,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1,1,0,0,0,1,0,1
5,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1,1,0,0,0,0,1,1
6,3692,38637,L Kumara,10928,SL,5,BOWL,46.78,8.5,35,1,1,52,1,0,0,1,0,0,1,1
7,3692,38637,Y Chahal,7910,IND,2,BOWL,54.21,8.5,35,1,1,39,1,0,0,1,0,1,0,1
8,3692,38637,B Kumar,1726,IND,2,BOWL,63.78,8.5,76,1,1,37,1,0,0,1,0,1,0,1
9,3692,38637,D Chameera,8393,SL,5,BOWL,72.85,9.0,30,1,1,31,1,0,0,1,0,0,1,1


In [ ]:
match2=sum(fdf.Points)
match2

687

Points for 2nd Match is 687

## Third T20 match Ind vs SL

In [ ]:
df = pd.read_csv('ind_sl_t20i_3rd_2022_02_27.csv')

In [ ]:
df = df.join(pd.get_dummies(df[['Role', 'Team']]))

In [ ]:
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

mo = pulp.LpProblem('Points maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Points'] for obj in df.index])

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) <= 7

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) <= 4

In [ ]:
mo.solve()

1

In [ ]:
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

In [ ]:
fdf = df[df['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

In [ ]:
fdf

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,3692,38642,S Iyer,9428,IND,2,BAT,88.80,8.5,196.0,2,1,100,0,1,0,0,1,0,1
1,3692,38642,C Asalanka,10934,SL,5,BAT,57.96,8.5,76.0,1,1,9,0,1,0,0,0,1,1
2,3692,38642,K Mishara,13886,SL,5,BAT,2.10,8.0,24.0,0,0,0,0,1,0,0,0,1,1
3,3692,38642,S Samson,8271,IND,2,WK,85.50,8.0,59.0,1,1,35,0,0,0,1,1,0,1
4,3692,38642,D Chandimal,6245,SL,5,WK,25.80,8.5,30.0,0,1,36,0,0,0,1,0,1,1
5,3692,38642,D Shanaka,8422,SL,5,AR,82.30,9.0,109.0,2,1,105,1,0,0,0,0,1,1
6,3692,38642,C Karunaratne,7952,SL,5,AR,14.50,8.5,31.0,0,1,47,1,0,0,0,0,1,1
7,3692,38642,D Hooda,9427,IND,2,AR,6.80,8.0,8.0,0,1,30,1,0,0,0,1,0,1
8,3692,38642,L Kumara,10928,SL,5,BOWL,60.44,8.5,87.0,2,1,60,0,0,1,0,0,1,1
9,3692,38642,R Bishnoi,14659,IND,2,BOWL,48.93,8.0,0.0,0,1,37,0,0,1,0,1,0,1


In [ ]:
match3=sum(fdf.Points)
match3

570

In [ ]:
np.mean((match1,match2,match3))

636.6666666666666

# Average points for series per match: 636.67